In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# 랜덤 시드 설정
np.random.seed(42)
random.seed(42)

# 현재 날짜를 기준으로 과거 데이터 생성
now = datetime.now()
end_date = now.strftime('%Y-%m-%d')
start_date = (now - timedelta(days=365)).strftime('%Y-%m-%d')
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# 상수 정의
PRODUCT_CATEGORIES = [
    '가구', '패브릭', '주방용품', '가전', '수납/정리', '조명', '인테리어소품',
    '생활용품', '반려동물', 'DIY/공구', '실내식물', '침구', '커튼/블라인드'
]

BRANDS = [
    '오늘의집', '이케아', '한샘', '까사미아', '리바트', '데코뷰', '무인양품', '니토리', 
    '루이스폴센', '시디즈', '자코모', '퍼시스', '보루네오', '에이스침대', '동서가구'
]

REGIONS = [
    '서울', '경기', '인천', '부산', '대구', '광주', '대전', '울산', '세종', 
    '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

PAYMENT_METHODS = [
    '신용카드', '체크카드', '무통장입금', '네이버페이', '카카오페이', '휴대폰결제', '간편결제'
]

STATUS = ['결제완료', '상품준비중', '배송중', '배송완료', '구매확정', '취소', '반품']

DELIVERY_TYPES = ['일반배송', '특급배송', '무료배송', '설치배송']

USER_SEGMENTS = ['loyal_customers', 'high_value_customers', 'new_customers', 'at_risk_customers', 'browsers']

# 데이터 생성 함수
def generate_sample_data(n_rows=1000):
    data = []
    
    # 사용자 ID 범위 (500명의 사용자)
    user_ids = range(1, 501)
    
    # 제품 ID 범위 (200개 제품)
    product_ids = range(1001, 1201)
    
    # 제품 정보 미리 생성
    products = {}
    for product_id in product_ids:
        category = random.choice(PRODUCT_CATEGORIES)
        brand = random.choice(BRANDS)
        
        # 카테고리별로 가격 범위 다르게 설정
        if category in ['가구', '가전']:
            base_price = np.random.randint(100000, 1000000)
        elif category in ['패브릭', '조명', '수납/정리']:
            base_price = np.random.randint(30000, 150000)
        else:
            base_price = np.random.randint(5000, 50000)
        
        product_name = f"{category} {brand} {product_id}"
        
        # 별점과 리뷰 수
        rating = round(max(1.0, min(5.0, np.random.normal(4.2, 0.5))), 1)
        review_count = int(np.random.exponential(scale=50)) + 1
        
        products[product_id] = {
            'product_name': product_name,
            'category': category,
            'brand': brand,
            'price': base_price,
            'rating': rating,
            'review_count': review_count
        }
    
    # 사용자 정보 미리 생성
    users = {}
    for user_id in user_ids:
        gender = random.choice(['여성', '남성', '미지정'])
        age_group = random.choice(['20대', '30대', '40대', '50대', '60대 이상'])
        region = random.choice(REGIONS)
        join_date = datetime.now() - timedelta(days=random.randint(1, 730))
        user_segment = random.choice(USER_SEGMENTS)
        
        users[user_id] = {
            'gender': gender,
            'age_group': age_group,
            'region': region,
            'join_date': join_date,
            'user_segment': user_segment
        }
    
    # 주문 데이터 생성
    for i in range(n_rows):
        order_id = f"ORDER-{i+10001}"
        user_id = random.choice(list(user_ids))
        product_id = random.choice(list(product_ids))
        
        # 날짜 선택 - 최근 날짜에 더 많은 주문이 있도록 가중치 부여
        days_ago = int(np.random.exponential(scale=30))
        days_ago = min(days_ago, 364)  # 최대 364일 전으로 제한
        order_date = now - timedelta(days=days_ago)
        
        # 사용자 정보
        user_info = users[user_id]
        
        # 제품 정보
        product_info = products[product_id]
        
        # 주문 수량
        quantity = np.random.choice([1, 1, 1, 2, 2, 3], p=[0.6, 0.15, 0.1, 0.1, 0.03, 0.02])
        
        # 할인율
        discount_rate = np.random.choice([0, 0, 0, 5, 10, 15, 20, 30]) / 100
        
        # 쿠폰 사용
        coupon_applied = random.choice([True, False])
        coupon_amount = round(product_info['price'] * random.choice([0.05, 0.1, 0.15, 0.2])) if coupon_applied else 0
        
        # 결제 수단
        payment_method = random.choice(PAYMENT_METHODS)
        
        # 배송 유형
        delivery_type = random.choice(DELIVERY_TYPES)
        
        # 할인 후 가격 계산
        discounted_price = product_info['price'] * (1 - discount_rate)
        total_price = discounted_price * quantity - coupon_amount
        
        # 주문 상태
        days_since_order = (now - order_date).days
        if days_since_order < 2:
            status = np.random.choice(['결제완료', '상품준비중', '배송중'], p=[0.3, 0.4, 0.3])
        elif days_since_order < 7:
            status = np.random.choice(['배송중', '배송완료', '구매확정'], p=[0.1, 0.3, 0.6])
        else:
            status = np.random.choice(['구매확정', '취소', '반품'], p=[0.9, 0.07, 0.03])
            
        # 리뷰 작성 여부
        review_written = False
        if status == '구매확정' and np.random.random() < 0.6:  # 60%의 확률로 리뷰 작성
            review_written = True
            
        # 장바구니에 담은 시간(분)
        cart_time_minutes = int(np.random.exponential(scale=15))
        
        # 배송까지 걸린 시간(일)
        if status in ['배송완료', '구매확정']:
            delivery_days = max(1, int(np.random.normal(3, 1)))
        else:
            delivery_days = None
            
        # 재구매 여부
        is_repurchase = False
        if user_id in user_ids and random.random() < 0.3:  # 30% 확률로 재구매
            is_repurchase = True
            
        # 판매자 ID
        seller_id = int(np.random.randint(1, 51))
        
        # 데이터 행 생성
        row = {
            'order_id': order_id,
            'user_id': user_id,
            'product_id': product_id,
            'product_name': product_info['product_name'],
            'category': product_info['category'],
            'brand': product_info['brand'],
            'order_date': order_date,
            'quantity': quantity,
            'unit_price': product_info['price'],
            'discount_rate': discount_rate,
            'coupon_applied': coupon_applied,
            'coupon_amount': coupon_amount,
            'total_price': total_price,
            'payment_method': payment_method,
            'delivery_type': delivery_type,
            'region': user_info['region'],
            'status': status,
            'review_written': review_written,
            'rating': product_info['rating'],
            'gender': user_info['gender'],
            'age_group': user_info['age_group'],
            'user_segment': user_info['user_segment'],
            'cart_time_minutes': cart_time_minutes,
            'delivery_days': delivery_days,
            'is_repurchase': is_repurchase,
            'seller_id': f"SELLER-{seller_id}"
        }
        
        data.append(row)
    
    # 데이터프레임 생성
    df = pd.DataFrame(data)
    
    # 날짜 열을 datetime 형식으로 변환
    df['order_date'] = pd.to_datetime(df['order_date'])
    
    return df

# 샘플 데이터 생성
sample_df = generate_sample_data(1000)

# CSV 파일로 저장
sample_df.to_csv('ohouse_sample_data.csv', index=False)

print(f"데이터가 생성되었습니다. 행 수: {len(sample_df)}, 열 수: {len(sample_df.columns)}")
print(f"파일명: ohouse_sample_data.csv")

데이터가 생성되었습니다. 행 수: 1000, 열 수: 26
파일명: ohouse_sample_data.csv
